<div style='background:#4a565e; padding:5px; margin:0'> 
    <div style=''>
        <img src="https://www.inegi.org.mx/img/home/logoinegibco.png" alt="logo inegi" />
    </div>
</div>

## Join information with the statistical

In [2]:
import requests
import os
import zipfile
import geopandas as gpd 
import pandas as pd

* Path dataset

In [3]:
path="../01-Data/01_data_geostatistical_grid"

* Geostatistical grid dataset

In [4]:
gdf_malla = gpd.read_file(path+"/mallaGeo.zip!INEGI_CPV2020_n9_.shp").to_crs(4326)

* Copernicus dataset

In [5]:
def CopernicusCPV(gdf_copernicus,field,new_field):
  interseccion = gdf_copernicus.sjoin(gdf_malla, how="left")
  resultado = interseccion.groupby("CODIGO")[field].apply(lambda x: x.value_counts().idxmax()).reset_index()
  resultado = resultado.rename(columns={field: new_field})
  return resultado

In [6]:
gdf1b = gpd.read_file(path+"/EMSR703_AOI01_GRA_MONIT01_v2.zip!EMSR703_AOI01_GRA_MONIT01_builtUpA_v2.shp")
gdf2b = gpd.read_file(path+"/EMSR703_AOI02_GRA_MONIT01_v2.zip!EMSR703_AOI02_GRA_MONIT01_builtUpA_v2.shp")
gdf_built = CopernicusCPV(gpd.GeoDataFrame(pd.concat([gdf1b, gdf2b], ignore_index=True)),"damage_gra","built")
gdf1f = gpd.read_file(path+"/EMSR703_AOI01_GRA_MONIT01_v2.zip!EMSR703_AOI01_GRA_MONIT01_observedEventA_v2.shp")
gdf2f = gpd.read_file(path+"/EMSR703_AOI02_GRA_MONIT01_v2.zip!EMSR703_AOI02_GRA_MONIT01_observedEventA_v2.shp")
gdf_flood = CopernicusCPV(gpd.GeoDataFrame(pd.concat([gdf1f, gdf2f], ignore_index=True)),"notation","flooded")
gdf1L = gpd.read_file(path+"/EMSR703_AOI01_GRA_MONIT01_v2.zip!EMSR703_AOI01_GRA_MONIT01_transportationL_v2.shp")
gdf2L = gpd.read_file(path+"/EMSR703_AOI02_GRA_MONIT01_v2.zip!EMSR703_AOI02_GRA_MONIT01_transportationL_v2.shp")
gdf_transL = CopernicusCPV(gpd.GeoDataFrame(pd.concat([gdf1L, gdf2L], ignore_index=True)),"damage_gra","transpor")
gdf1p = gpd.read_file(path+"/EMSR703_AOI01_GRA_MONIT01_v2.zip!EMSR703_AOI01_GRA_MONIT01_transportationP_v2.shp")
gdf2p = gpd.read_file(path+"/EMSR703_AOI02_GRA_MONIT01_v2.zip!EMSR703_AOI02_GRA_MONIT01_transportationP_v2.shp")
gdf_transp = CopernicusCPV(gpd.GeoDataFrame(pd.concat([gdf1p, gdf2p], ignore_index=True)),"damage_gra","transpor")

In [7]:
gdf_trans = gpd.GeoDataFrame(pd.concat([gdf_transL, gdf_transp], ignore_index=True))
gdf_Copern = gdf_built.merge(gdf_flood, on='CODIGO', how='outer').merge(gdf_trans, on='CODIGO', how='outer')
gdf_Copern

,CODIGO,built,flooded,transpor
0,O275585310,Destroyed,NaN,NaN
1,O275585313,Destroyed,NaN,NaN
2,O275585316,Damaged,NaN,NaN
3,O275585317,Damaged,NaN,NaN
4,O275585318,Damaged,NaN,NaN
...,...,...,...,...
931,O283602035,NaN,Flooded area,NaN
932,O283602060,NaN,Flooded area,NaN
933,O283602071,NaN,Flooded area,Possibly damaged
934,O283602080,NaN,Flooded area,Possibly damaged


* DENUE dataset

In [8]:
%time gdf_DENUE = gpd.read_file(path+"/denue_12_shp.zip!conjunto_de_datos/denue_inegi_12_.shp").to_crs(4326)

CPU times: user 1min 46s, sys: 1.12 s, total: 1min 47s
Wall time: 1min 46s


In [9]:
interseccionDENUE = gpd.sjoin(gdf_DENUE, gdf_malla, how="left")
interseccionDENUE['sec']  = interseccionDENUE['codigo_act'].astype(str).str[:2]
inter = interseccionDENUE[['sec', 'CODIGO', 'per_ocu','clee']]
gdf_DENUEcounts = inter.groupby(['CODIGO', 'sec']).size().reset_index(name='count_sec')
gdf_DENUEsec = gdf_DENUEcounts.pivot(index='CODIGO', columns='sec', values='count_sec')
gdf_DENUEsec = gdf_DENUEsec.fillna(0)

gdf_DENUEcountp = inter.groupby(['CODIGO', 'per_ocu']).size().reset_index(name='count_per')
gdf_DENUEper = gdf_DENUEcountp.pivot(index='CODIGO', columns='per_ocu', values='count_per')
gdf_DENUEper = gdf_DENUEper.fillna(0)

gdf_DENUEsecPer = gdf_DENUEsec.merge(gdf_DENUEper, on='CODIGO')
gdf_DENUEtot = inter.groupby(['CODIGO'])['clee'].count().reset_index(name='Tot_DENUE')
gdf_DENUEres = gdf_DENUEsecPer.merge(gdf_DENUEtot, on='CODIGO', how='outer')
gdf_DENUEres

,CODIGO,11,21,22,23,31,32,33,43,46,...,81,93,0 a 5 personas,101 a 250 personas,11 a 30 personas,251 y más personas,31 a 50 personas,51 a 100 personas,6 a 10 personas,Tot_DENUE
0,O272876641,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1
1,O272876754,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1
2,O275117002,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,2
3,O275126362,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4,O275126415,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3930,O283740060,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,2
3931,O283740063,0.0,0.0,1.0,0.0,3.0,0.0,1.0,3.0,20.0,...,5.0,1.0,42.0,0.0,2.0,0.0,0.0,0.0,3.0,47
3932,O283740066,0.0,0.0,0.0,0.0,5.0,1.0,0.0,2.0,19.0,...,4.0,0.0,36.0,0.0,0.0,0.0,0.0,0.0,2.0,38
3933,O283740300,0.0,0.0,0.0,0.0,3.0,0.0,1.0,0.0,2.0,...,2.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,1.0,8


* Charter activations dataset

In [10]:
gdf_Flooding = gpd.read_file(path+"/vap-974-1-product.geojson")
intersecFlooding = gpd.sjoin(gdf_Flooding, gdf_malla, how="left")
df_CA_Flooding = intersecFlooding.groupby(['CODIGO'])['DN'].count().reset_index(name='CA_flood')
df_CA_Flooding

,CODIGO,CA_flood
0,O275585271,7
1,O275585272,12
2,O275585283,10
3,O275585284,5
4,O275585285,14
...,...,...
1052,O283602075,438
1053,O283602076,202
1054,O283602077,70
1055,O283602080,408


In [13]:
gdf_Slides = gpd.read_file(path+"/GeoJSON-file-for-Landslides-Mexico.geojson")
intersecSlides = gpd.sjoin(gdf_Slides, gdf_malla, how="left")
df_CA_Slides = intersecSlides.groupby(['CODIGO'])['DN'].count().reset_index(name='CA_Slides')
df_CA_Slides

,CODIGO,CA_Slides
0,O275582516,21
1,O275585221,230
2,O275585222,89
3,O275585224,193
4,O275585225,125
...,...,...
1219,O283602074,814
1220,O283602075,285
1221,O283602076,38
1222,O283602077,390


## Join data with the geostatistical grid
The result of the union of the data sets is saved in a new Shapefile file

In [14]:
for columna in gdf_malla.columns:
    gdf_malla[columna] = gdf_malla[columna].replace(-6, 0)
ret_res1= gdf_malla.merge(gdf_Copern, on='CODIGO', how='outer')
ret_res2= ret_res1.merge(gdf_DENUEres, on='CODIGO', how='outer')
ret_res3= ret_res2.merge(df_CA_Flooding, on='CODIGO', how='outer')
ret_res4= ret_res3.merge(df_CA_Slides, on='CODIGO', how='outer')
ret_res4.to_file("01_grid/grid_integration.shp", encoding='utf-8', truncate_colnames=False)

/tmp/ipykernel_2339/3435957281.py:7: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  ret_res4.to_file("01_grid/grid_integration.shp", encoding='utf-8', truncate_colnames=False)


In [15]:
ret_res4

,ID,CODIGO,POBTOT,P_0A2,POB0_14,P15A29A,P30A59A,P_3A5,P_6A11,P_12A14,...,0 a 5 personas,101 a 250 personas,11 a 30 personas,251 y más personas,31 a 50 personas,51 a 100 personas,6 a 10 personas,Tot_DENUE,CA_flood,CA_Slides
0,486885,O283610011,81,6,27,20,25,6,11,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,483055,O283387346,8,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,337047,O272887270,17,3,6,4,3,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,336366,O272887155,235,6,82,64,65,12,50,14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,332410,O272873635,21,0,7,4,5,0,4,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47871,374337,O283032416,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47872,484859,O283388817,221,19,69,64,68,15,17,18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47873,388530,O283146043,55,0,16,13,17,4,5,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47874,439677,O283307464,477,43,184,136,115,29,73,39,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<div style='background:#4a565e; padding:5px; margin:0'> 
    <img src="https://cienciadedatos.inegi.org.mx/laboratorio/img/icono_lcid.jpg" alt="LCiD" width="30"/>
    <div style="color:#ccd6de; float:right" >
        Laboratorio de ciencia de datos y métodos modernos de Producción de información |
        <a href="mailto:lcid@inegi.org.mx">lcid@inegi.org.mx</a>
    </div>
</div>